# Integer minimum cost flows with separable convex cost objective

The Problem of finding the most likely mulitpath split for a payment pair on the lightning network can be reduced to an **integer minimum cost flows with separable convex cost objective** and yields a polynomial solver c.f.: https://twitter.com/renepickhardt/status/1385144337907044352

This notebook tries to implement a solution given in the Textbook Network Flows Network Flows: Theory, Algorithms, and Applications by Ravindra K. Ahuja, Thomas L. Magnanti and James B. Orlin

This approach follows mainly chapter 9, 10.2 and 14.5 of the Textbook. 

Other good resources are the Lecture series http://courses.csail.mit.edu/6.854/20/ by David Karger (http://people.csail.mit.edu/karger/) with lecture notes at http://courses.csail.mit.edu/6.854/current/Notes/ and recorded videos at: https://www.youtube.com/channel/UCtv9PiQVUDzsT4yl7524DCg/videos

Other good resources are the lecture nodes by Hochbaum: https://hochbaum.ieor.berkeley.edu/

I did not find an open source implementation. Code for similar problems seems to be part of this library: https://github.com/frangio68/Min-Cost-Flow-Class but it seems to only allow quadratic cost functions. 

Following previous research ( https://arxiv.org/abs/2103.08576 ) our cost function is f(x) = log(c) - log(c-x) which turns out to be convex (f'(x) = 1/(c-x) and f"(x) = 1/(c-x)^2 from which we can see that f"(x) > 0)

In [1]:
import json
import networkx as nx
import math

log = math.log2

In [2]:
def import_channel_graph():
    # retrieve this by: lightning-cli listchannels > listchannels.json
    f = open("listchannels.json")
    jsn = json.load(f)
    G = nx.Graph()
    for channel in jsn["channels"]:
        src = channel["source"]
        dest = channel["destination"]
        cap = int(int(channel["satoshis"])/1000)
        sid = channel["short_channel_id"]
        G.add_edge(src,dest,cap=cap,sid=sid)
    return G

channel_graph = import_channel_graph()

G = nx.DiGraph()
G.add_edge("S","A",cap=2)
G.add_edge("S","X",cap=1)
G.add_edge("A","B",cap=2)
G.add_edge("X","B",cap=9)
G.add_edge("X","Y",cap=7)
G.add_edge("Y","D",cap=4)
G.add_edge("B","D",cap=4)

def generate_null_flow(G):
    flow = {n:{} for n in G.nodes()}
    for u,v in G.edges():
        flow[u][v]=0
        flow[v][u]=0
    return flow

def next_hop(path):
    for i in range(1,len(path)):
        src = path[i-1]
        dest = path[i]
        yield (src,dest)
        
def augment_path(flow,path,amt):
    for src,dest in next_hop(path):
        flow[src][dest]+=amt

def cost(x,c):
    return log(c+1)-log(c+1-x)

SRC = "03efccf2c383d7bf340da9a3f02e2c23104a0e4fe8ac1a880c8e2dc92fbdacd9df"
DEST = "022c699df736064b51a33017abfc4d577d133f7124ac117d3d9f9633b6297a3b6a"
FLOW = 9200

In [3]:


def compute_delta_residual_network(G,x,delta,C=cost):
    residual = nx.DiGraph()
    for i, j in G.edges():
        #print(i,j)
        cap = G[i][j]["cap"]
        f = x[i][j]
        if x[i][j] == 0 and x[j][i]==0: # no flow:             
            if delta<=cap:
                residual.add_edge(i,j,weight=(C(f + delta, cap) - C(f,cap))/delta)
                residual.add_edge(j,i,weight=(C(f + delta, cap) - C(f,cap))/delta)
        if x[i][j] > 0 and x[j][i] > 0:
            print("FLOW IN BOTH DIRECTION CAN'T / SHOULDN'T HAPPEN",i,j, x[i][j],x[j][i])
        if x[i][j]>0:
            if f+delta <=cap:
                residual.add_edge(i,j,weight=(C(f + delta, cap) - C(f,cap))/delta)
            if delta <= f:#backward flow
                residual.add_edge(j,i,weight=(C(f - delta, cap) - C(f,cap))/delta)
        else: # if flow was in other direction enter in the other direction
            f=x[j][i]
            if f+delta <=cap:
                residual.add_edge(j,i,weight=(C(f + delta, cap) - C(f,cap))/delta)
            if delta <= f:#backward flow
                residual.add_edge(i,j,weight=(C(f - delta, cap) - C(f,cap))/delta)
        
        ## DEBUG
        if residual.has_edge(i,j):
            print(i,j,residual[i][j]["weight"])
        if residual.has_edge(j,i):
            print(j,i,residual[j][i]["weight"])
    return residual
    
#compute_delta_residual_network(channel_graph,x,10)

In [4]:
def apply_reduced_cost(R,pi):
    for i,j in R.edges():
        R[i][j]["rc"]=R[i][j]["weight"]+pi[i]-pi[j]
        print(i,j,R[i][j]["rc"])

In [5]:
def cpacity_scaling(s,d,U,G):
    x = generate_null_flow(G)
    pi = {n:0 for n in G.nodes}
    e = {n:0 for n in G.nodes}
    e[s]=U
    e[d]=-U
    delta = 2**int(log(U))
    while delta >= 1:
        print("{}-scaling phase".format(delta))
        R = compute_delta_residual_network(G,x,delta)
        
        for i,j in R.edges():
            #test and correct reduced cost optimality condition
            if R[i][j]["weight"]+pi[i]-pi[j] <0:
                #might be too simplisic as flow in oposite direction might exist which we would have to decrease
                print("correct reduced cost condition: ",i,j,R[i][j]["weight"],pi[i],pi[j])
                if x[j][i]>=delta:
                    x[j][i]-=delta
                else:
                    x[i][j]+=delta
                e[i]-=delta
                e[j]+=delta
        print(e)
        print(x) 
        R = compute_delta_residual_network(G,x,delta)
        S = [n for n,k in e.items() if k>=delta]
        T = [n for n,k in e.items() if k<=-delta]
        print(S,T)
        while len(S) >0 and len(T)>0:
            k=S[-1]
            l=T[-1]
            S=S[:-1]
            T=T[:-1]
            #path = nx.dijkstra_path(R,k,l)
            apply_reduced_cost(R,pi)
            distances, paths = nx.single_source_dijkstra(R,k,weight="rc")
            #distances, paths = nx.single_source_bellman_ford(R,k,weight="rc")
            for n, d_value in distances.items():
                print(n,d_value)
                pi[n]-=d_value
            path = paths[l]
            augment_path(x,path,delta)
            print("augmented path:", path, "with:", delta)
            e[k]-=delta
            e[l]+=delta
            print(e)
            print(x)
        delta = int(delta/2)
    
#cpacity_scaling(SRC,DEST, FLOW, channel_graph)
cpacity_scaling("S","D", 2, G)


2-scaling phase
S A 0.792481250360578
A S 0.792481250360578
A B 0.792481250360578
B A 0.792481250360578
X B 0.1609640474436811
B X 0.1609640474436811
X Y 0.20751874963942196
Y X 0.20751874963942196
B D 0.36848279708310305
D B 0.36848279708310305
Y D 0.36848279708310305
D Y 0.36848279708310305
{'S': 2, 'A': 0, 'X': 0, 'B': 0, 'Y': 0, 'D': -2}
{'S': {'A': 0, 'X': 0}, 'A': {'S': 0, 'B': 0}, 'X': {'S': 0, 'B': 0, 'Y': 0}, 'B': {'A': 0, 'X': 0, 'D': 0}, 'Y': {'X': 0, 'D': 0}, 'D': {'B': 0, 'Y': 0}}
S A 0.792481250360578
A S 0.792481250360578
A B 0.792481250360578
B A 0.792481250360578
X B 0.1609640474436811
B X 0.1609640474436811
X Y 0.20751874963942196
Y X 0.20751874963942196
B D 0.36848279708310305
D B 0.36848279708310305
Y D 0.36848279708310305
D Y 0.36848279708310305
['S'] ['D']
S A 0.792481250360578
A S 0.792481250360578
A B 0.792481250360578
B A 0.792481250360578
B X 0.1609640474436811
B D 0.36848279708310305
X B 0.1609640474436811
X Y 0.20751874963942196
Y X 0.20751874963942196
Y D 0

ValueError: ('Contradictory paths found:', 'negative weights?')

In [55]:
log(2)

1.0